In [ ]:
!pip install tqdm
from IPython.display import display, clear_output
from PIL import Image
import matplotlib.pyplot as plt
import time
from tqdm.auto import tqdm
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

#readme
print("Welcome to the ANS Test")
time.sleep(2)
clear_output(wait=False)
print("Please read:\n" 
      "We wish to record your response data\n"
      "to an anonymized public data repository.\n"
      "Your data will be used for educational teaching purposes,\n"
      "practicing data analysis and visualization.\n"
      "Please type 'yes' in the box below if you consent to the upload.")

result = input("> ")

if result == "yes":
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")

else:
    raise(Exception("User did not consent to continue test."))

# Name 
id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:

- two letters based on the initials (first and last name) of a childhood friend

- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise

then your unique identifer would be CBTC

"""

print(id_instructions)

user_id = input("> ")

print("User entered id:", user_id)

gender = input("What is your gender")

age = input("What is your age")


# Answer dictionary
participant_answers = {
    'participant_id': user_id,
    'gender': gender,
    'age': age,
    'q1_nl': 12, 'q1_nr': 9, 'q1_ans': '',
    'q2_nl': 16, 'q2_nr': 12, 'q2_ans': '',
    'q3_nl': 20, 'q3_nr': 15, 'q3_ans': '',
    'q4_nl': 14, 'q4_nr': 12, 'q4_ans': '',
    'q5_nl': 21, 'q5_nr': 18, 'q5_ans': '',
    'q6_nl': 18, 'q6_nr': 16, 'q6_ans': '',
    'q7_nl': 10, 'q7_nr': 9, 'q7_ans': '',
    'q8_nl': 20, 'q8_nr': 18, 'q8_ans': '',
    'q9_nl': 9, 'q9_nr': 12, 'q9_ans': '',
    'q10_nl': 12, 'q10_nr': 16, 'q10_ans': '',
    'q11_nl': 15, 'q11_nr': 20, 'q11_ans': '',
    'q12_nl': 12, 'q12_nr': 14, 'q12_ans': '',
    'q13_nl': 18, 'q13_nr': 21, 'q13_ans': '',
    'q14_nl': 16, 'q14_nr': 18, 'q14_ans': '',
    'q15_nl': 9, 'q15_nr': 10, 'q15_ans': '',
    'q16_nl': 18, 'q16_nr': 20, 'q16_ans': '',
    'q17_nl': 12, 'q17_nr': 9, 'q17_ans': '',
    'q18_nl': 118, 'q18_nr': 12, 'q18_ans': '',
    'q19_nl': 20, 'q19_nr': 15, 'q19_ans': '',
    'q20_nl': 14, 'q20_nr': 12, 'q20_ans': '',
    'q21_nl': 21, 'q21_nr': 18, 'q21_ans': '',
    'q22_nl': 18, 'q22_nr': 16, 'q22_ans': '',
    'q23_nl': 10, 'q23_nr': 9, 'q23_ans': '',
    'q24_nl': 20, 'q24_nr': 18, 'q24_ans': '',
    'q25_nl': 9, 'q25_nr': 12, 'q25_ans': '',
    'q26_nl': 12, 'q26_nr': 16, 'q26_ans': '',
    'q27_nl': 15, 'q27_nr': 20, 'q27_ans': '',
    'q28_nl': 12, 'q28_nr': 14, 'q28_ans': '',
    'q29_nl': 18, 'q29_nr': 21, 'q29_ans': '',
    'q30_nl': 16, 'q30_nr': 18, 'q30_ans': '',
    'q31_nl': 9, 'q31_nr': 10, 'q31_ans': '',
    'q32_nl': 18, 'q32_nr': 20, 'q32_ans': '',
}

# Image list
images = ['12v9.png', '16v12.png', '20v15.png', '14v12.png', '21v18.png', '18v16.png', '10v9.png', '20v18.png', '9v12.png', 
          '12v16.png', '15v20.png', '12v14.png', '18v21.png', '16v18.png', '9v10.png', '18v20.png', 
          '12v9.png', '16v12.png', '20v15.png', '14v12.png', '21v18.png', '18v16.png', '10v9.png', '20v18.png', '9v12.png', 
          '12v16.png', '15v20.png', '12v14.png', '18v21.png', '16v18.png', '9v10.png', '18v20.png']

# Collect responses
def show_image_get_response(image_path):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
    time.sleep(0.75)
    clear_output(wait=True)
    
    print(f'Which side has more dots? (L/R) You have 3 seconds to respond.')
    start_time = time.time()
    response = ''
    while time.time() - start_time < 3:
        response = input()
        if response:
            break
    clear_output(wait=True)
    
    time.sleep(1.5)
    
    return response
    
responses = []
for image in tqdm(images, desc='Running ANS Test'):
    response = show_image_get_response(image)
    responses.append(response)

# Update dict.
for i, response in enumerate(responses):
    participant_answers[f'q{i+1}_ans'] = response

#helper code
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

#sends to form
form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSft3WVq4me5sDoP7uUbERuPWMatzkf4PqqyprRqnbCW_0vpAQ/viewform'
send_to_google_form(participant_answers, form_url)



Which side has more dots? (L/R) You have 3 seconds to respond.
